# Demostración Modelos de Clasificación y Regresión
## Data Mining - Doctorado UDP 2024
**Bastián González-Bustamante** \
Octubre 2024

In [6]:
## Importar librerías necesarias
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, KBinsDiscretizer
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, SVR
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import (
    mean_squared_error, mean_absolute_error, r2_score,
    accuracy_score, precision_score, recall_score, f1_score, classification_report
)
import warnings

In [7]:
## Ignorar warnings para una salida más limpia
warnings.filterwarnings('ignore')

## Cargar el dataset (California Housing)

In [8]:
california = fetch_california_housing()
X = pd.DataFrame(california.data, columns=california.feature_names)
y = pd.Series(california.target, name='MedHouseVal')  # Valor medio de las viviendas

In [9]:
# Inspección
print(X.head())

   MedInc  HouseAge  AveRooms  AveBedrms  Population  AveOccup  Latitude  \
0  8.3252      41.0  6.984127   1.023810       322.0  2.555556     37.88   
1  8.3014      21.0  6.238137   0.971880      2401.0  2.109842     37.86   
2  7.2574      52.0  8.288136   1.073446       496.0  2.802260     37.85   
3  5.6431      52.0  5.817352   1.073059       558.0  2.547945     37.85   
4  3.8462      52.0  6.281853   1.081081       565.0  2.181467     37.85   

   Longitude  
0    -122.23  
1    -122.22  
2    -122.24  
3    -122.25  
4    -122.25  


In [5]:
# Inspección
print(y.head())

0    4.526
1    3.585
2    3.521
3    3.413
4    3.422
Name: MedHouseVal, dtype: float64


## División de los datos en entrenamiento y prueba

In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=86
)

In [11]:
## Escalado de características (para mejorar rendimiento de SVM)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [12]:
## Preparación para clasificación
## Discretizar la variable objetivo en 3 categorías: Bajo, Medio, Alto
discretizer = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='quantile')
y_train_class = discretizer.fit_transform(y_train.values.reshape(-1, 1)).astype(int).flatten()
y_test_class = discretizer.transform(y_test.values.reshape(-1, 1)).astype(int).flatten()

In [13]:
## Función para imprimir métricas de clasificación
def print_classification_metrics(y_true, y_pred, model_name):
    print(f"--- {model_name} ---")
    print(f"Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"Precision: {precision_score(y_true, y_pred, average='weighted'):.4f}")
    print(f"Recall: {recall_score(y_true, y_pred, average='weighted'):.4f}")
    print(f"F1-Score: {f1_score(y_true, y_pred, average='weighted'):.4f}")
    print("Classification Report:")
    print(classification_report(y_true, y_pred))
    print("\n")

* **Accuracy (exactitud)**.  Fracción de predicciones correctas sobre el total de predicciones realizadas. El accuracy puede ser engañoso, ya que un modelo que simplemente predice la clase mayoritaria puede obtener un alto accuracy sin ser un buen modelo.
* **Precision (precisión)**. Fracción de instancias correctamente clasificadas como positivas entre todas las clasificadas como positivas. Es importante cuando el costo de las falsas alarmas (falsos positivos) es alto.
* **Recall (Sensibilidad)**. Fracción de instancias positivas correctamente clasificadas por el modelo. Es importante cuando queremos minimizar los falsos negativos (no pasar por alto casos positivos).
* **F1-Score**. El F1-Score equilibra la precisión y el recall. Es útil cuando se necesita un equilibrio entre ambas métricas, y es particularmente relevante en conjuntos de datos desequilibrados donde un modelo.

In [14]:
## Función para imprimir métricas de regresión
def print_regression_metrics(y_true, y_pred, model_name):
    print(f"--- {model_name} ---")
    print(f"MSE: {mean_squared_error(y_true, y_pred):.4f}")
    print(f"RMSE: {np.sqrt(mean_squared_error(y_true, y_pred)):.4f}")
    print(f"MAE: {mean_absolute_error(y_true, y_pred):.4f}")
    print(f"R² Score: {r2_score(y_true, y_pred):.4f}")
    print("\n")

* **Mean Squared Error (MSE) - Error Cuadrático Medio**. Promedio de los cuadrados de los errores o diferencias entre los valores reales y las predicciones del modelo. Cuanto más pequeño es el MSE, mejor es el ajuste del modelo.
* **Root Mean Squared Error (RMSE) - Raíz del Error Cuadrático Medio.** Es la raíz cuadrada del MSE. Un RMSE más bajo indica un mejor rendimiento.
* **Mean Absolute Error (MAE) - Error Absoluto Medio.** Promedio de las diferencias absolutas entre los valores reales y las predicciones. Un MAE más bajo indica que las predicciones del modelo están más cerca de los valores reales.
* **R² Score (Coeficiente de Determinación).** Representa la proporción de la varianza total explicada por el modelo. 

## Naive Bayes (Clasificación)

In [15]:
nb = GaussianNB()
nb.fit(X_train_scaled, y_train_class)
y_pred_nb = nb.predict(X_test_scaled)

In [16]:
# Resultados Naive Bayes
print_classification_metrics(y_test_class, y_pred_nb, "Naive Bayes Clasificación")

--- Naive Bayes Clasificación ---
Accuracy: 0.5719
Precision: 0.6315
Recall: 0.5719
F1-Score: 0.5629
Classification Report:
              precision    recall  f1-score   support

           0       0.54      0.84      0.66      1414
           1       0.46      0.43      0.45      1327
           2       0.89      0.43      0.58      1387

    accuracy                           0.57      4128
   macro avg       0.63      0.57      0.56      4128
weighted avg       0.63      0.57      0.56      4128





## Support Vector Machine (SVM, Regresión)

In [17]:
## SVM para Regresión
svr = SVR(kernel='rbf')
svr.fit(X_train_scaled, y_train)
y_pred_svr = svr.predict(X_test_scaled)
print_regression_metrics(y_test, y_pred_svr, "SVM Regresión")

--- SVM Regresión ---
MSE: 0.3232
RMSE: 0.5685
MAE: 0.3853
R² Score: 0.7611




## Support Vector Machine (SVM, Clasificación)

In [18]:
## SVM para Clasificación
svc = SVC(kernel='rbf', probability=True)
svc.fit(X_train_scaled, y_train_class)
y_pred_svc = svc.predict(X_test_scaled)
print_classification_metrics(y_test_class, y_pred_svc, "SVM Clasificación")

--- SVM Clasificación ---
Accuracy: 0.7602
Precision: 0.7660
Recall: 0.7602
F1-Score: 0.7624
Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.78      0.80      1414
           1       0.63      0.69      0.66      1327
           2       0.83      0.80      0.82      1387

    accuracy                           0.76      4128
   macro avg       0.76      0.76      0.76      4128
weighted avg       0.77      0.76      0.76      4128





## Random Forest (Regresión)

In [19]:
## Random Forest para regresión con 100 árboles
rf_reg_100 = RandomForestRegressor(n_estimators=100, random_state=86)
rf_reg_100.fit(X_train, y_train)
y_pred_rf_reg_100 = rf_reg_100.predict(X_test)
print_regression_metrics(y_test, y_pred_rf_reg_100, "Random Forest Regresión (100 árboles)")

--- Random Forest Regresión (100 árboles) ---
MSE: 0.2511
RMSE: 0.5011
MAE: 0.3283
R² Score: 0.8144




In [20]:
## Random Forest para regresión con 200 árboles
rf_reg_200 = RandomForestRegressor(n_estimators=200, random_state=86)
rf_reg_200.fit(X_train, y_train)
y_pred_rf_reg_200 = rf_reg_200.predict(X_test)
print_regression_metrics(y_test, y_pred_rf_reg_200, "Random Forest Regresión (200 árboles)")

--- Random Forest Regresión (200 árboles) ---
MSE: 0.2484
RMSE: 0.4984
MAE: 0.3263
R² Score: 0.8164




## Random Forest (Clasificación)

In [21]:
## Random Forest para clasificación con 100 árboles
rf_class_100 = RandomForestClassifier(n_estimators=100, random_state=86)
rf_class_100.fit(X_train, y_train_class)
y_pred_rf_class_100 = rf_class_100.predict(X_test)
print_classification_metrics(y_test_class, y_pred_rf_class_100, "Random Forest Clasificación (100 árboles)")

--- Random Forest Clasificación (100 árboles) ---
Accuracy: 0.8171
Precision: 0.8170
Recall: 0.8171
F1-Score: 0.8169
Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.88      0.86      1414
           1       0.73      0.73      0.73      1327
           2       0.87      0.84      0.85      1387

    accuracy                           0.82      4128
   macro avg       0.82      0.82      0.82      4128
weighted avg       0.82      0.82      0.82      4128





In [22]:
## Random Forest para clasificación con 200 árboles
rf_class_200 = RandomForestClassifier(n_estimators=200, random_state=86)
rf_class_200.fit(X_train, y_train_class)
y_pred_rf_class_200 = rf_class_200.predict(X_test)
print_classification_metrics(y_test_class, y_pred_rf_class_200, "Random Forest Clasificación (200 árboles)")

--- Random Forest Clasificación (200 árboles) ---
Accuracy: 0.8181
Precision: 0.8177
Recall: 0.8181
F1-Score: 0.8177
Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.88      0.86      1414
           1       0.74      0.73      0.73      1327
           2       0.86      0.84      0.85      1387

    accuracy                           0.82      4128
   macro avg       0.82      0.82      0.82      4128
weighted avg       0.82      0.82      0.82      4128





## XGBoost (Regresión)

In [23]:
## XGBoost para Regresión
xg_reg = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=200, random_state=86)
xg_reg.fit(X_train, y_train)
y_pred_xg_reg = xg_reg.predict(X_test)
print_regression_metrics(y_test, y_pred_xg_reg, "XGBoost Regresión")

--- XGBoost Regresión ---
MSE: 0.2151
RMSE: 0.4638
MAE: 0.3066
R² Score: 0.8410




## XGBoost (Clasificación)

In [24]:
## XGBoost para Clasificación
xg_class = xgb.XGBClassifier(n_estimators=200, use_label_encoder=False, eval_metric='mlogloss', random_state=86)
xg_class.fit(X_train_scaled, y_train_class)
y_pred_xg_class = xg_class.predict(X_test_scaled)
print_classification_metrics(y_test_class, y_pred_xg_class, "XGBoost Clasificación")

--- XGBoost Clasificación ---
Accuracy: 0.8358
Precision: 0.8370
Recall: 0.8358
F1-Score: 0.8362
Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.88      0.87      1414
           1       0.75      0.77      0.76      1327
           2       0.89      0.86      0.87      1387

    accuracy                           0.84      4128
   macro avg       0.84      0.83      0.83      4128
weighted avg       0.84      0.84      0.84      4128



